# model

> Fill in a module description here

In [ ]:
#| default_exp sketch_transformer.model

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
import math
from typing import Optional, Tuple

import torch
import torch.nn as nn
from torch.nn import functional as F

from singleline_models.sketch_transformer.masks import *
from singleline_models.sketch_transformer.layers import *

In [ ]:
#| export
class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_ff, max_seq_len, dropout_rate):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = nn.Linear(5, d_model)
        self.pos_encoding = PositionalEncoding(d_model, dropout_rate, max_len=max_seq_len)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, max_seq_len=max_seq_len, dropout_prob=dropout_rate) for _ in range(num_layers)])

    def forward(self, x, mask):
        seq_len = x.shape[1]
        x = self.embedding(x) # (batch, seq, d_model)
        x *= torch.sqrt(torch.tensor(self.d_model))
        # add positional embedding, and apply dropout
        x = self.pos_encoding(x)

        for i, layer in enumerate(self.enc_layers):
            x = layer(x, mask)
        return x # (batch, seq, d_model)

In [ ]:
#| export
class Decoder(nn.Module):

    def __init__(self, num_layers, d_model, num_heads, d_ff,
                    max_seq_len=1000,
                    dropout_rate=0.1):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = nn.Linear(5, d_model)
        self.pos_encoding = PositionalEncoding(d_model, dropout_rate, max_len=max_seq_len)

        self.dec_layers = nn.ModuleList([DecoderLayer(d_model=d_model, num_heads=num_heads, d_ff=d_ff, max_seq_len=max_seq_len, dropout_prob=dropout_rate)
                            for _ in range(num_layers)])

    def forward(self, x, enc_output, padding_mask, dec_target_padding_mask, look_ahead_mask):
        seq_len = x.shape[1]
        x = self.embedding(x) # (batch, seq, d_model)
        x *= torch.sqrt(torch.tensor(self.d_model))
        # add positional embedding, and apply dropout
        x = self.pos_encoding(x)

        attention_weights = {}
        for i, layer in enumerate(self.dec_layers):
            x, block1, block2 = layer(x, enc_output, padding_mask, dec_target_padding_mask, look_ahead_mask)
            attention_weights['decoder_layer{}_block1'.format(i + 1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i + 1)] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

In [ ]:
#| export
class ReconstructionLoss(nn.Module):
   
    def forward(self, pred, real):
        pred_locations = pred[:, :, :2]
        real_locations = real[:, :, :2]
        location_loss = F.mse_loss(pred_locations, real_locations, reduction='none') # [batch_size, n_seq, 2]
        location_loss = torch.mean(location_loss, dim=2) # [batch_size, n_seq]

        pred_metadata = pred[:, :, 2:] # un-normalized logits
        real_metadata = real[:, :, 2:] # true labels in probability spaces (add up to 1).
        # for K-dimensional inputs, torch cross_entropy() requires:
        # - batch as first dimension,
        # - class as second,
        # - ... other dimensions
        pred_metadata = pred_metadata.transpose(2, 1) # [batch_size, n_classes, n_seq]
        real_idx = torch.argmax(real_metadata.transpose(2, 1), dim=1) # [batch_size, n_seq]
        metadata_loss = F.cross_entropy(pred_metadata, real_idx, reduction='none') # [batch_size, n_seq]

        # final dimension is "end-of-sequence" - invert it to get a mask for valid parts of the sequence
        mask = torch.abs(real[..., -1]-1) # [batch_size, n_seq] 

        # slightly less efficient to mask & average these parts of the loss separately, but this allows
        # logging them separately during training runs.
        location_loss *= mask
        location_loss = torch.mean(location_loss)
        metadata_loss *= mask
        metadata_loss = torch.mean(metadata_loss)

        loss = location_loss + metadata_loss
        
        return loss, dict(location_loss=location_loss, metadata_loss=metadata_loss)

In [ ]:
#| export
class Model(nn.Module):
    def __init__(self, hp):
        super().__init__()
        self.hp = hp
        self.encoder = Encoder(
            num_layers=hp.n_layer,
            d_model=hp.d_model,
            num_heads=hp.n_head,
            d_ff=hp.d_ff,
            max_seq_len=hp.max_seq_length+2,
            dropout_rate=hp.dropout_rate)
        self.bottleneck_layer = SelfAttn(d_model=hp.d_model, d_lowerdim=hp.d_lowerdim)
        self.expand_layer = DenseExpander(in_dim=hp.d_lowerdim, out_dim=hp.d_model, seq_len=hp.max_seq_length+2)
        self.decoder = Decoder(num_layers=hp.n_layer,
            d_model=hp.d_model,
            num_heads=hp.n_head,
            d_ff=hp.d_ff,
            max_seq_len=hp.max_seq_length+2,
            dropout_rate=hp.dropout_rate)
        self.output_layer = nn.Linear(hp.d_model, 5)
    
    def encode(self, x, mask):
        enc_output = self.encoder(x, mask)
        lowerdim_output, _ = self.bottleneck_layer(enc_output)
        return lowerdim_output, enc_output
    
    def decode(self, embedding, target, dec_padding_mask, dec_target_padding_mask, look_ahead_mask):
        padding_mask = torch.zeros_like(dec_padding_mask).bool() if self.hp.blind_decoder_mask else dec_padding_mask
        pre_decoder = self.expand_layer(embedding)
        
        dec_output, attention_weights = self.decoder(
            x=target,
            enc_output=pre_decoder,
            padding_mask=padding_mask,
            dec_target_padding_mask=dec_target_padding_mask,
            look_ahead_mask=look_ahead_mask)
        final_output = self.output_layer(dec_output)
        return final_output, attention_weights

    def forward(self, input_seq, target_seq, enc_padding_mask, dec_padding_mask, dec_target_padding_mask, look_ahead_mask):
        lowerdim_output, enc_output = self.encode(input_seq, enc_padding_mask)
        
        final_output, attention_weights = self.decode(
            embedding=lowerdim_output,
            target=target_seq,
            dec_padding_mask=dec_padding_mask,
            dec_target_padding_mask=dec_target_padding_mask,
            look_ahead_mask=look_ahead_mask)
        
        return final_output, lowerdim_output #, enc_output, attention_weights

    def encode_from_seq(self, inp_seq):
        enc_padding_mask = create_padding_mask(inp_seq)
        self.eval()
        with torch.no_grad():
            res = self.encode(inp_seq, enc_padding_mask)
            return res

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()